In [4]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
#mixed = base
#mixed = base.extend(target)
mixed = np.concatenate((base,target))

scaler = MinMaxScaler()
mixed_scaled = scaler.fit_transform(mixed)

base_scaled = mixed_scaled[:15]
#base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = mixed_scaled[15:20]
#target_unscaled = scaler.inverse_transform(target_scaled)
#print(mixed)
# scaler = MinMaxScaler()
# base_scaled = scaler.fit_transform(base)
# base_unscaled = scaler.inverse_transform(base_scaled)

# target_scaled = scaler.fit_transform(target[:5])
# target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
num_attr = mixed_scaled[0].size
print(target_scaled)

[[ 0.1496063 ]
 [ 0.22834646]
 [ 0.38582677]
 [ 0.93700787]
 [ 0.16535433]]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [5]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[ 0.21496063]
 [ 0.04611924]
 [ 0.41207349]]
0.0245101252107


In [6]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(num_clusters):
        val = []
        for j in range(num_attr):
            val.append(alpha[j]*base_centroids[i][j] + beta[j])
        
        target_centroids.append(val)
             

    target_centroids = np.array(target_centroids)
    print("new centroids: ", target_centroids)
    return target_centroids
    #print(target_centroids)

In [26]:
def kmeans(k,dataItems,centroids,maxIter,num_attr):
    
    #print(k,dataItems,centroids,maxInter,num_attr)
    old_centroids = []
    groups = []
    
    for i in range(k):
        groups.append([])
        
    iter = 0
    while(iter < maxIter):
        old_centroids = centroids
        for item in dataItems:
            row = []
            for centroid in centroids:
                diff = 0
                for i in range(num_attr):
                    diff += abs(item[i] - centroid[i])
                    
                row.append(diff)
                
            idx = row.index(min(row))
            groups[idx].append(item)
                    
            
        iter += 1
        
    ss = 0    
    for i in range(k):
        for item in groups[i]:
            for j in range(num_attr):
            
                diff = abs(item[j]- old_centroids[i][j])
                ss += pow(diff,2)
            
        
    return (groups,ss)
        

In [27]:
def cal_gradient(clusters,reframed_centroids,old_alpha,old_beta):
    gradient_alpha = 0
    gradient_beta = 0
    for i in range(0,num_clusters):
        gradient = 0
        #print(reframed_centroids[i])
        #print(centroids[i])
        for member in clusters[i]:
            #print("ghghh")
            #print(member[0])
            gradient =  (reframed_centroids[i][0] - member[0])
            gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
            gradient_beta = gradient_beta + gradient

#         gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
#         gradient_beta = gradient_beta + gradient
#         print(clusters[i])
#         print(gradient_alpha)
#         print(gradient_beta)
        print("******")
    print(gradient_alpha,gradient_beta)    
    new_alpha = old_alpha-.1*gradient_alpha
    new_beta = old_beta-.1*gradient_beta
    return [new_alpha,new_beta]

In [28]:
sum_m = 0
sum_d = 0
for i in range(len(base_centroids)):
    sum_m = sum_m + base_centroids[i][0]
    
avg_m = sum_m/len(base_centroids)

for i in range(len(target_scaled)):
    sum_d = sum_d + target_scaled[i][0]

print(sum_m,sum_d)
avg_d = sum_d/len(target_scaled)

alpha = avg_m/avg_d
beta = 0
print(alpha)
        


0.673153355831 1.86614173228
0.601198848034


In [29]:
reframed_centroids =  cal_centroids(alpha,beta)
km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
#kmeans.fit(target_scaled)
best_error = km[1]
#centroids = kmeans.cluster_centers_
#labels = kmeans.labels_

count = 0
best_alpha = alpha
best_beta = beta
while(1):
    #print(centroids)
    print("best error: ",best_error)
    
    #clusters = find_members(centroids,labels)
    
    #reframed_centroids = closest_centroids(reframed_centroids,centroids)
    
    new_alphabeta = cal_gradient(km[0],reframed_centroids,alpha,beta)
    
    alpha = new_alphabeta[0]
    beta = new_alphabeta[1]
    
    print("new alpha beta", alpha, beta)
    
    reframed_centroids =  cal_centroids(alpha,beta)
    
    km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
    #kmeans.fit(target_scaled)
    new_error = km[1]
    
    print("compare ",best_error,new_error)
    if(new_error < best_error):
        best_alpha = alpha
        best_beta = beta
        best_error = new_error
        count = 0
        
    elif(new_error == best_error):
        if(count<5):
            count += 1
            continue
        else:
            break;
                
    else:
        break; 

    #base_centroids = kmeans.cluster_centers_
    #labels = kmeans.labels_

    #print(kmeans.cluster_centers_)
    #print(old_error,new_error)
    
print("finalparameters", best_alpha,best_beta)

new centroids:  [[ 0.24773811]
 [ 0.12923408]
 [ 0.02772683]]
best error:  0.49625702502
******
******
******
-0.207464912752 -0.864459242173
new alpha beta 0.621945339309 0.0864459242173
new centroids:  [[ 0.34273311]
 [ 0.22013969]
 [ 0.11512957]]
compare  0.49625702502 0.358798080132
best error:  0.358798080132
******
******
******
-0.230005545051 -0.730276688513
new alpha beta 0.644945893815 0.159473593069
new centroids:  [[ 0.4252387 ]
 [ 0.29811157]
 [ 0.189218  ]]
compare  0.358798080132 0.267130586478
best error:  0.267130586478
******
******
******
-0.196257704649 -0.448010321338
new alpha beta 0.664571664279 0.204274625202
new centroids:  [[ 0.47812699]
 [ 0.34713137]
 [ 0.23492416]]
compare  0.267130586478 0.224231364538
best error:  0.224231364538
******
******
******
-0.194903600327 -0.336110886626
new alpha beta 0.684062024312 0.237885713865
new centroids:  [[ 0.51976954]
 [ 0.38493212]
 [ 0.26943413]]
compare  0.224231364538 0.200968138825
best error:  0.200968138825
***

******
******
******
-0.084542665322 0.0131239974348
new alpha beta 1.36037446291 0.167591012652
new centroids:  [[ 0.72816527]
 [ 0.46001796]
 [ 0.23033044]]
compare  0.0609738149212 0.0598617863585
best error:  0.0598617863585
******
******
******
-0.0839264752867 0.0130328252161
new alpha beta 1.36876711044 0.16628773013
new centroids:  [[ 0.73032037]
 [ 0.46051877]
 [ 0.22941422]]
compare  0.0598617863585 0.0587727328166
best error:  0.0587727328166
******
******
******
-0.083300742804 0.0129400764252
new alpha beta 1.37709718472 0.164993722488
new centroids:  [[ 0.73245897]
 [ 0.4610154 ]
 [ 0.22850439]]
compare  0.0587727328166 0.0577064495115
best error:  0.0577064495115
******
******
******
-0.0826658499494 0.012845808977
new alpha beta 1.38536376972 0.16370914159
new centroids:  [[ 0.73458083]
 [ 0.46150781]
 [ 0.22760106]]
compare  0.0577064495115 0.056662722622
best error:  0.056662722622
******
******
******
-0.0820221797648 0.0127500810727
new alpha beta 1.39356598769 0.16

new alpha beta 1.91944047888 0.0797758462604
new centroids:  [[ 0.87072638]
 [ 0.49237998]
 [ 0.16829897]]
compare  0.0197262135427 0.0197106094425
best error:  0.0197106094425
******
******
******
-0.0117127662528 0.00186155208852
new alpha beta 1.9206117555 0.0795896910516
new centroids:  [[ 0.87102288]
 [ 0.4924456 ]
 [ 0.16816684]]
compare  0.0197106094425 0.019695585046
best error:  0.019695585046
******
******
******
-0.0114964424822 0.00182726159926
new alpha beta 1.92176139975 0.0794069648917
new centroids:  [[ 0.87131389]
 [ 0.49251001]
 [ 0.16803713]]
compare  0.019695585046 0.0196811193354
best error:  0.0196811193354
******
******
******
-0.0112838497206 0.00179355911005
new alpha beta 1.92288978473 0.0792276089807
new centroids:  [[ 0.87159952]
 [ 0.49257321]
 [ 0.16790982]]
compare  0.0196811193354 0.0196671920261
best error:  0.0196671920261
******
******
******
-0.0110749336288 0.00176043618924
new alpha beta 1.92399727809 0.0790515653617
new centroids:  [[ 0.87187984]


new centroids:  [[ 0.8853127 ]
 [ 0.49559971]
 [ 0.1617824 ]]
compare  0.0193143392263 0.0193142786774
best error:  0.0193142786774
******
******
******
-0.000739851065156 0.000117875286213
new alpha beta 1.97717982208 0.0705880046256
new centroids:  [[ 0.8853314 ]
 [ 0.49560382]
 [ 0.16177403]]
compare  0.0193142786774 0.019314220481
best error:  0.019314220481
******
******
******
-0.000725349636927 0.000115565238881
new alpha beta 1.97725235704 0.0705764481017
new centroids:  [[ 0.88534973]
 [ 0.49560786]
 [ 0.16176581]]
compare  0.019314220481 0.0193141645456
best error:  0.0193141645456
******
******
******
-0.000711131390941 0.000113300287752
new alpha beta 1.97732347018 0.0705651180729
new centroids:  [[ 0.8853677 ]
 [ 0.49561182]
 [ 0.16175776]]
compare  0.0193141645456 0.0193141107835
best error:  0.0193141107835
******
******
******
-0.000697190838183 0.000111079559261
new alpha beta 1.97739318927 0.070554010117
new centroids:  [[ 0.88538532]
 [ 0.4956157 ]
 [ 0.16174987]]
co

-3.34588297884e-05 5.33157667168e-06
new alpha beta 1.98070673347 0.0700260430873
new centroids:  [[ 0.88622278]
 [ 0.49580001]
 [ 0.16137472]]
compare  0.0193127852249 0.0193127851061
best error:  0.0193127851061
******
******
******
-3.28006601923e-05 5.22669981881e-06
new alpha beta 1.98071001354 0.0700255204173
new centroids:  [[ 0.88622361]
 [ 0.49580019]
 [ 0.16137435]]
compare  0.0193127851061 0.0193127849919
best error:  0.0193127849919
******
******
******
-3.21554353167e-05 5.12388562932e-06
new alpha beta 1.98071322908 0.0700250080287
new centroids:  [[ 0.88622442]
 [ 0.49580037]
 [ 0.16137399]]
compare  0.0193127849919 0.0193127848822
best error:  0.0193127848822
******
******
******
-3.15229006517e-05 5.02309355019e-06
new alpha beta 1.98071638137 0.0700245057194
new centroids:  [[ 0.88622522]
 [ 0.49580055]
 [ 0.16137363]]
compare  0.0193127848822 0.0193127847768
best error:  0.0193127847768
******
******
******
-3.09028066886e-05 4.92428382412e-06
new alpha beta 1.980719

new centroids:  [[ 0.88626192]
 [ 0.49580862]
 [ 0.16135719]]
compare  0.0193127821908 0.0193127821902
best error:  0.0193127821902
******
******
******
-2.3345803205e-06 3.72011747768e-07
new alpha beta 1.98086183252 0.070001328378
new centroids:  [[ 0.88626198]
 [ 0.49580864]
 [ 0.16135716]]
compare  0.0193127821902 0.0193127821896
best error:  0.0193127821896
******
******
******
-2.28864948038e-06 3.64692743227e-07
new alpha beta 1.98086206138 0.0700012919087
new centroids:  [[ 0.88626204]
 [ 0.49580865]
 [ 0.16135714]]
compare  0.0193127821896 0.0193127821891
best error:  0.0193127821891
******
******
******
-2.24362227895e-06 3.57517732225e-07
new alpha beta 1.98086228574 0.070001256157
new centroids:  [[ 0.88626209]
 [ 0.49580866]
 [ 0.16135711]]
compare  0.0193127821891 0.0193127821885
best error:  0.0193127821885
******
******
******
-2.19948093887e-06 3.50483881389e-07
new alpha beta 1.98086250569 0.0700012211086
new centroids:  [[ 0.88626215]
 [ 0.49580867]
 [ 0.16135708]]
c

******
******
-1.05193750164e-07 1.67624688296e-08
new alpha beta 1.98087294108 0.0699995582457
new centroids:  [[ 0.88626479]
 [ 0.49580925]
 [ 0.1613559 ]]
compare  0.0193127821754 0.0193127821754
best error:  0.0193127821754
******
******
******
-1.03124131175e-07 1.64326779961e-08
new alpha beta 1.98087295139 0.0699995566024
new centroids:  [[ 0.88626479]
 [ 0.49580925]
 [ 0.1613559 ]]
compare  0.0193127821754 0.0193127821754
best error:  0.0193127821754
******
******
******
-1.01095230516e-07 1.61093757223e-08
new alpha beta 1.9808729615 0.0699995549915
new centroids:  [[ 0.88626479]
 [ 0.49580925]
 [ 0.1613559 ]]
compare  0.0193127821754 0.0193127821754
best error:  0.0193127821754
******
******
******
-9.91062472255e-08 1.57924340272e-08
new alpha beta 1.98087297141 0.0699995534122
new centroids:  [[ 0.88626479]
 [ 0.49580926]
 [ 0.1613559 ]]
compare  0.0193127821754 0.0193127821754
best error:  0.0193127821754
******
******
******
-9.71563957311e-08 1.54817282327e-08
new alpha 

In [30]:
target_cent = cal_centroids(best_alpha,best_beta)

new centroids:  [[ 0.88626491]
 [ 0.49580928]
 [ 0.16135585]]


In [31]:
target_sc = mixed_scaled[15:]
# base_unscaled = scaler.inverse_transform(base_scaled)
# target_unscaled = scaler.inverse_transform(target_sc)
# print(target_sc)
# print(base_scaled)
# print(base_unscaled)
# print(target_unscaled)

In [36]:
kmeans = KMeans(n_clusters=num_clusters, init=target_cent, max_iter=1)
kmeans.fit(target_sc)
print(kmeans.inertia_)

0.0993797035213


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [37]:
kmeans = KMeans(n_clusters=num_clusters, init=base_centroids, max_iter=1)
kmeans.fit(target_sc)
print(kmeans.inertia_)

0.337106729769


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [7]:
from collections import Counter

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

c = Counter(labels)

clusters = []
for i in range(0,3):
    clusters.append([])
    
#print(kmeans.cluster_centers_)
for i in range(len(target_scaled)):
    #print("coordinate:",X[i], "label:", labels[i], "keyword:", keywords[i])
    clusters[labels[i]].append(target_scaled[i])

#unscaled_centroids = scaler.inverse_transform(centroids)
gradient_alpha = 0
gradient_beta = 0
for i in range(0,3):
    gradient = 0
    print(reframed_centroids[i][0])
    print(centroids[i])
    for member in clusters[i]:
        print(member[0])
        gradient = reframed_centroids[i][0] - member[0]
    
    gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
    gradient_beta = gradient_beta + gradient
    print(clusters[i])
    print(gradient_alpha)
    print(gradient_beta)
    print("******")

0.8143006871988968
[1.]
1.0
[array([1.])]
-0.15121507802630121
-0.1856993128011032
******
0.17470606057119187
[0.105]
0.0
0.10000000000000003
0.30000000000000004
0.020000000000000018
[array([0.]), array([0.1]), array([0.3]), array([0.02])]
-0.12418699163742009
-0.03099325222991134
******
1.5609915493312188
[1.]
[]
-0.12418699163742009
-0.03099325222991134
******


In [8]:
alpha = alpha-1*gradient_alpha
beta = beta-1*gradient_beta
print(alpha,beta)

1.8542031036350763 0.03099325222991134


In [9]:
reframed_centroids =  cal_centroids(alpha,beta)
kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_
print(old_error)

[[0.90374747]
 [0.21824036]
 [1.70403858]]
0.03845


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
